In [2064]:
import pandas as pd
import numpy as np
import re

In [2065]:
mn_or_bk = 'mn' #bk or mn
year = '1880'
#doubttype = 'not so confident' #doubt or not so confident
doubttype = 'not so confident' #doubt or not so confident

In [2066]:
df = pd.read_csv('/Users/prajwal/Desktop/Columbia/C4SR/' + mn_or_bk + '_' + year + '/after cleaning/' + mn_or_bk + '_' + year + '_after_cleaning_' + doubttype + '4.csv')


In [2067]:

df.columns = ['index', 'Full Address', 'H House Number', 
              'H Street Name', 'c_street1', 'c_street2', 'n_street1', 'n_street2', 
              'new h city', 'new h house number', 'new h street name', 'unidentified h info']
"""
df.columns = ['index', 'n_street1', 'n_street2','c_street1', 'c_street2', 'Full Address', 
              'H House Number', 
              'H Street Name', 
              'new h house number', 'new h street name','new h city', 'unidentified h info']
"""

"\ndf.columns = ['index', 'n_street1', 'n_street2','c_street1', 'c_street2', 'Full Address', \n              'H House Number', \n              'H Street Name', \n              'new h house number', 'new h street name','new h city', 'unidentified h info']\n"

In [2068]:
df = df.drop(['index'], axis=1)

In [2069]:
df.head()

,Full Address,H House Number,H Street Name,c_street1,c_street2,n_street1,n_street2,new h city,new h house number,new h street name,unidentified h info
0,H ANNS AVE N 153,NaN,NaN,NaN,NaN,ANNS AVE,153,NaN,NaN,NaN,NaN
1,H N 169 3 AVE.,169.0,N 3 AVE.,NaN,NaN,169,3 AVE.,NaN,169.0,N 3,RD AVE
2,H 3 AVE N 166,NaN,NaN,NaN,NaN,3 AVE,166,NaN,NaN,166,3 AVE N
3,H N 150 COURTLAND AVE.,150.0,N COURTLAND AVE.,NaN,NaN,150,COURTLAND AVE.,NaN,150.0,COURTLAND,AVE N
4,H N 148 3 AVE.,148.0,N 3 AVE.,NaN,NaN,148,3 AVE.,NaN,148.0,N 3,RD AVE


In [2070]:
# Combines data from various columns into two: hn_1 (for house number) and best_match (for street name)

In [2071]:
hn_1 = {}
best_match = {}

for i in range(0,len(df)):
    hn_1[i] = df['new h street name'][i]
    best_match[i] = df['unidentified h info'][i]

for i in range(0,len(df)):
#for i in range(35,36):
    #print('begin',hn_1[i],best_match[i])
    
    #handle corner cases
    if pd.isna(df['c_street1'][i]) == False or pd.isna(df['c_street2'][i]) == False:
        #print('1',i)
        try:
            best_match[i] = str(df['c_street1'][i]) + ' & ' + str(df['c_street2'][i])
        except:
            if pd.isna(df['c_street1'][i]) == False:
                best_match[i] = str(df['c_street1'])
            else:
                best_match[i] = str(df['c_street2'])
                
    #handle near cases
    elif pd.isna(df['n_street1'][i]) == False or pd.isna(df['n_street2'][i]) == False:
        #print('2',i)
        try:
            best_match[i] = str(df['n_street1'][i]) + ' & ' + str(df['n_street2'][i])
        except:
            if pd.isna(df['n_street1'][i]) == False:
                best_match[i] = str(df['n_street1'])
            else:
                best_match[i] = str(df['n_street2'])
    
    #if corner and near cases don't have any data, use H House Number and H Street Name
    elif pd.isna(hn_1[i]) == True or pd.isna(best_match[i]) == True:
        #print('3',i)
        try:
            hn_1[i] = df['H House Number'][i]
        except:
            pass
        try:
            best_match[i] = df['H Street Name'][i]
        except:
            pass
    #if everything above didn't work, use new h house number and new h street name
    elif pd.isna(hn_1[i]) == True or pd.isna(best_match[i]) == True:
        #print('4',i)
        try:
            hn_1[i] = df['new h house number'][i]
        except:
            pass
        try:
            best_match[i] = df['new h street name'][i]
        except:
            pass
    
    #if everything above still didn't work, use new h city for best_match
    elif pd.isna(df['new h city'][i]) == False and pd.isna(best_match[i]) == True:
        #print('5',i)
        best_match[i] = df['new h city'][i]
    
    # return a blank house number and Full Address for best_match if nothing above worked
    else:
        if pd.isna(hn_1[i]) == True and pd.isna(best_match[i]) == True:
            #print('6',i)
            hn_1[i] = ''
            best_match[i] = df['Full Address'][i]
    
    """
    elif pd.isna(df['H House Number'][i]) == False or pd.isna(df['H Street Name'][i]) == False:
        print('3',i)
        try:
            hn_1[i] = df['H House Number'][i]
        except:
            pass
        try:
            best_match[i] = df['H Street Name'][i]
        except:
            pass
    
    elif pd.isna(df['new h house number'][i]) == False or pd.isna(df['new h street name'][i]) == False:
        print('4',i)
        try:
            hn_1[i] = df['new h house number'][i]
        except:
            pass
        try:
            best_match[i] = df['new h street name'][i]
        except:
            pass
    
    elif pd.isna(df['new h city'][i]) == False:
        print('5',i)
        best_match[i] = df['new h city'][i]
    
    else:
        print('6',i)
        hn_1[i] = ''
        best_match[i] = df['Full Address'][i]
    """
    #if pd.isna(df['H House Number'][i]) == True and pd.isna(df['new h house number'][i]) == True:
        #print('7',i)
        #hn_1[i] = ''
    
    if pd.isna(hn_1[i]) == True:
        #print('7',i)
        hn_1[i] = ''




In [2072]:
#df.head(20)

In [2073]:
df2 = pd.DataFrame([list(hn_1.values()), list(best_match.values())])

In [2074]:
df2 = df2.T


In [2075]:
df2.columns = ['hn_1','best_match']

In [2076]:
def find_number(text):
    num = re.findall(r'[0-9]+',str(text))
    return " ".join(num)
df2['hn_1']=df2['hn_1'].apply(lambda x: find_number(x))

In [2077]:
#df2

In [2078]:
for i in range(0, len(df2)):
    if df2['hn_1'][i] == '':
        try:
            df2['hn_1'][i] = df['H House Number'][i]
        except:
            pass

In [2079]:
for i in range(0, len(df2)):
    if pd.isna(df2['best_match'][i]) == True:
        try:
            df2['best_match'][i] = df['unidentified h info'][i]
        except:
            pass

In [2080]:
for i in range(0, len(df2)):
    if len(str(df2['best_match'][i])) <=2:
        if pd.isna(df['H House Number'][i])==False:
            hn_1[i] = df['H House Number'][i]
            df2['best_match'][i] = df['H Street Name'][i]
        else:
            df2['best_match'][i] = df['Full Address'][i]

In [2081]:
for i in range(0, len(df2)):
    if pd.isna(df2['hn_1'][i]) == True:
        if pd.isna(df['H House Number'][i])==False:
            hn_1[i] = df['H House Number'][i]
            df2['best_match'][i] += ' ' + df['H Street Name'][i]
        elif pd.isna(df['new h house number'][i])==False:
            df2['hn_1'][i] = df['new h house number'][i]
            df2['best_match'][i] = str(df2['best_match'][i]) + ' ' + df['new h street name'][i]

In [2082]:
"""
for i in range(0, len(df2)):
    if df2['hn_1'][i] == '':
        try:
            df2['hn_1'] = df['new h house number'][i] + df['new h street name'][i]
        except:
            pass
"""

"\nfor i in range(0, len(df2)):\n    if df2['hn_1'][i] == '':\n        try:\n            df2['hn_1'] = df['new h house number'][i] + df['new h street name'][i]\n        except:\n            pass\n"

In [2083]:
for i in range(0, len(df2)):
    try:
        df2['hn_1'][i] = df2['hn_1'][i].replace(' 0','')
    except:
        pass
    
    try:
        if df2['best_match'][i].index('&') >=0:
            df2['hn_1'][i] = ''
    except:
        pass
    
    try:
        if df2['best_match'][i].index('nan &') >=0:
            df2['best_match'][i] = df2['best_match'][i].replace('nan &','')
    except:
        pass

In [2084]:
unidentified_h_is_H = df.loc[df['unidentified h info']=='H']
for i in unidentified_h_is_H.index:
    df2.loc[i]['hn_1'] = unidentified_h_is_H.loc[i]['new h house number']
    df2.loc[i]['best_match'] = unidentified_h_is_H.loc[i]['new h street name']

In [2085]:
for i in range(0, len(df2)):
    if df2['best_match'][i] == 'nan':
        print(i)
        check1 = pd.isna(df['H House Number'][i])
        check2 = pd.isna(df['H Street Name'][i])
        check3 = pd.isna(df['new h city'][i])
        check4 = pd.isna(df['new h house number'][i])
        check5 = pd.isna(df['new h street name'][i])
        check6 = pd.isna(df['unidentified h info'][i])
        combine = ''
        if check1 == False:
            combine += str(df['H House Number'][i])
        elif check2 == False:
            combine += str(df['H Street Name'][i])
        elif check3 == False:
            combine += str(df['new h city'][i])
        elif check4 == False:
            combine += str(df['new h house number'][i])
        elif check5 == False:
            combine += str(df['new h street name'][i])
        elif check6 == False:
            combine += str(df['unidentified h info'][i])
        if len(combine) > 0:
            print(i, combine)
            df2['best_match'][i] = combine
        else:
            df2['best_match'][i] = df['Full Address'][i]

In [2086]:
for i in range(0, len(df)):
    if pd.isna(df['c_street1'][i]) == False and pd.isna(df['c_street2'][i]) == False:
        df2['best_match'][i] = df['c_street1'][i] + ' & ' + df['c_street2'][i]

for i in range(0, len(df)):
    if pd.isna(df['n_street1'][i]) == False and pd.isna(df['n_street2'][i]) == False:
        df2['best_match'][i] = df['n_street1'][i] + ' & ' + df['n_street2'][i]

In [2087]:
for i in range(0, len(df2)):
    if pd.isna(df2['best_match'][i]) == True:
        df2['best_match'][i] = df['Full Address'][i]

In [2088]:
for i in range(0, len(df2)):
    try:
        if df2['best_match'][i].index('AVE G') >= 0 and df2['best_match'][i].index('OWANUS')>=0:
            index_of_ave_g = df2['best_match'][i].index('AVE G')
            df2['best_match'][i] = df2['best_match'][i].replace('AVE G','AVE ')
            df2['best_match'][i] = df2['best_match'][i].replace('OWANUS','GOWANUS')
    except:
        pass

In [2089]:
for i in range(0, len(df2)):
    if df2['hn_1'][i] == df['new h street name'][i]:
        df2['best_match'][i] = df['unidentified h info'][i]

In [2090]:
for i in range(0, len(df)):
    try:
        if str(df['new h street name'][i]).index('AVE')>=0:
            print(i, df['new h street name'][i])
    except:
        pass

11 AVE A
26 AVE A
158 AVE H
170 AVE H
178 AVE A
389 AVE C
431 AVE H
457 AVE A
495 AVE B
528 AVE A
598 AVE A
623 AVE A
702 AVE H
712 AVE A
759 AVE A
915 AVE A
919 AVE A
955 AVE C
999 AVE A
1060 AVE A
1070 AVE H
1219 AVE H
1245 AVE A
1268 AVE A
1318 AVE A
1319 AVE A
1323 AVE A
1355 AVE C
1366 AVE C
1443 AVE A
1445 AVE A
1487 AVE A
1558 AVE B
1646 AVE C
1683 AVE B
1769 AVE C
1817 AVE A
1906 AVE C
1911 AVE C
1937 AVE H
1952 AVE A
1975 AVE H
2051 AVE A
2129 AVE A
2136 AVE A
2166 AVE C
2171 AVE A
2180 AVE C
2182 AVE H
2186 AVE H
2231 AVE A
2306 AVE C
2310 AVE C
2325 AVE C
2347 AVE C
2386 AVE A
2400 AVE C
2403 AVE C
2466 AVE A
2473 AVE A
2512 AVE C
2543 AVE C
2556 AVE A
2569 AVE A
2570 AVE A
2574 AVE C
2605 AVE E
2620 BEAVER
2668 AVE H
2695 AVE A
2706 AVE H
2731 AVE A
2778 AVE C
2779 AVE C
2797 AVE A
2856 AVE C
2877 AVE C
2899 AVE C
2914 AVE C
2974 AVE A
3052 AVE A
3092 AVE A
3120 AVE C
3161 AVE H
3171 AVE C
3203 AVE C
3284 AVE A
3415 AVE C
3430 AVE C
3456 AVE A
3479 AVE A
3483 AVE A
3497 AVE

In [2091]:
#df['Full Address'][42]

In [2092]:
for i in range(0, len(df)):
    if isinstance(df['new h house number'][i], float) and pd.isna(df['new h house number'][i]) == False:
        df2['hn_1'][i] = df['new h house number'][i]
        df2['best_match'][i] = df['new h street name'][i]

In [2093]:
df[350:500]

,Full Address,H House Number,H Street Name,c_street1,c_street2,n_street1,n_street2,new h city,new h house number,new h street name,unidentified h info
350,H. MO.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,MO.
351,H. CANADA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CANADA
352,W H N 161 10 AVE.,161.0,N 10 AVE.,NaN,NaN,W 161,10 AVE.,NaN,161.0,N 10,W TAV
353,H. GA.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,GA.
354,H W 45I 35,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,W 35,45I
355,"3 PINE AND 1130 BROADWAY,H N WASHN 173",NaN,NaN,NaN,NaN,WASHN,173,NaN,NaN,173,N WASHN
356,H. BKLYN,NaN,NaN,NaN,NaN,NaN,NaN,BKLYN,NaN,NaN,NaN
357,H. BKLYN,NaN,NaN,NaN,NaN,NaN,NaN,BKLYN,NaN,NaN,NaN
358,H. S) SUYDARH AVE. J. C.,NaN,NaN,NaN,NaN,NaN,NaN,J. C.,NaN,NaN,S) SUYDARAV.
359,H 235UFFOLK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,235,UFFOLK


In [2094]:
df2[350:500]

,hn_1,best_match
350,NaN,MO.
351,NaN,CANADA
352,161,N 10
353,NaN,GA.
354,35,45I
355,,WASHN & 173
356,NaN,H. BKLYN
357,NaN,H. BKLYN
358,NaN,S) SUYDARAV.
359,235,UFFOLK


In [2095]:
for i in range(0, len(df2)):
    if (any(c.isalpha() for c in str(df2['hn_1'][i])) == True 
    and pd.isna(df2['hn_1'][i]) == False
    and pd.isna(df2['best_match'][i]) == False):
        print(i, df2['hn_1'][i])

In [2096]:
"""
for i in range(0, len(df2)):
    try:
        if df2['best_match'][i].index('AVE G')>=0:
            print(i, df2['best_match'][i])
    except:
        pass
"""

"\nfor i in range(0, len(df2)):\n    try:\n        if df2['best_match'][i].index('AVE G')>=0:\n            print(i, df2['best_match'][i])\n    except:\n        pass\n"

In [2097]:
df2.to_csv('/Users/prajwal/Desktop/Columbia/C4SR/' + mn_or_bk + '_' + year + '/for address builder/' + mn_or_bk + '_' + year + '_' + doubttype + '_ready4.csv')
